In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

%matplotlib inline  

KeyboardInterrupt: 

In [ ]:
# set data input folder
# download files described in README.md to this folder
input_folder = "inputs_data"

# set output folder
output_folder = "outputs"
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

# define p-value to use for significance
p_value = 0.001

# list files to be queried for significant features
file_list = ["20200805_A549_WG_Screen_guide_normalized_feature_select_merged_median_ALLBATCHES___CP186___ALLWELLS.csv.gz",
"20210422_6W_CP257_guide_normalized_feature_select_median_merged_ALLBATCHES___DMEM___ALLWELLS.csv.gz",
"20210422_6W_CP257_guide_normalized_feature_select_median_merged_ALLBATCHES___HPLM___ALLWELLS.csv.gz"]

In [ ]:
# load barcodes used in experiment
guide_df = pd.read_csv('../common_files/Barcodes.csv')
# guides with other dialouts were not used in this experiment
guide_df = guide_df.query('dialout == 1 | dialout ==3')
guide_list = list(guide_df['sgRNA'])

,Metadata_Foci_Barcode_MatchedTo_GeneCode,Metadata_Foci_Barcode_MatchedTo_Barcode,Cells_AreaShape_CentralMoment_0_1,Cells_AreaShape_CentralMoment_0_3,Cells_AreaShape_CentralMoment_1_0,Cells_AreaShape_CentralMoment_1_2,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,Cells_AreaShape_HuMoment_1,...,Nuclei_Texture_SumVariance_ConA_5_01_256,Nuclei_Texture_SumVariance_DAPI_Painting_10_01_256,Nuclei_Texture_SumVariance_DAPI_Painting_10_03_256,Nuclei_Texture_SumVariance_Mito_5_01_256,Nuclei_Texture_SumVariance_Phalloidin_10_01_256,Nuclei_Texture_SumVariance_Phalloidin_10_03_256,Nuclei_Texture_SumVariance_Phalloidin_5_01_256,Nuclei_Texture_SumVariance_WGA_10_01_256,Nuclei_Texture_SumVariance_WGA_10_03_256,Nuclei_Texture_SumVariance_WGA_5_03_256
0,A1BG,CAAGAGAAAGACCACGAGCA,-0.085682,-0.007318,0.048388,0.005510,-0.029950,0.45247,0.128870,-0.251250,...,0.26915,-0.510610,-0.81353,-0.037740,-0.090246,-0.078726,-0.084498,-0.013743,0.06056,-0.093787
1,A1BG,CATCTTCTTTCACCTGAACG,0.009774,-0.003006,0.049408,0.006071,0.059289,-0.73213,-0.477380,-0.192550,...,-0.36389,-0.159780,-0.33425,-0.028623,-0.190650,-0.066117,-0.126050,-0.298190,-0.26774,-0.203280
2,A1BG,CTCCGGGGAGAACTCCGGCG,-0.058185,0.016190,-0.128340,-0.010445,-0.198250,0.11875,0.325120,-0.185930,...,-0.55675,0.168100,-0.09446,-0.088934,-0.256140,-0.121030,-0.277940,-0.344150,-0.29566,-0.349900
3,A1BG,TGGAAGTCCACTCCACTCAG,-0.097838,-0.008437,0.116660,0.004938,0.229590,0.60542,0.599460,-0.172990,...,-0.46291,0.406220,0.10339,-0.082164,-0.243800,-0.209550,-0.144330,-0.347550,-0.34362,-0.309940
4,A1CF,AGTTATGTTAGGTATACCCG,-0.057318,-0.029085,-0.034795,-0.026508,0.422990,0.37453,0.081628,0.045344,...,-0.28838,-0.023266,0.43572,-0.002229,0.076596,0.139300,0.128120,-0.069559,-0.23150,-0.108940


In [ ]:
# Extract expression data from the Broad Institute Dependency Map
A549_express = pd.read_csv(os.path.join(input_folder,"CCLE_expression_A549.csv"))
for i in range(len(A549_express.index)):
    A549_express.iloc[i,0] = A549_express.iloc[i,0].split()[0]
A549_express_min , A549_express_max = A549_express['A549_LUNG'].min(), A549_express['A549_LUNG'].max()
A549_express = A549_express.sort_values(by = ['Unnamed: 0']).set_index('Unnamed: 0')

# Extract expression data from the Broad Institute Dependency Map
HELA_express = pd.read_csv(os.path.join(input_folder,"CCLE_expression_HELA.csv"))
for i in range(len(HELA_express.index)):
    HELA_express.iloc[i,0] = HELA_express.iloc[i,0].split()[0]
HELA_express_min , HELA_express_max = HELA_express['HELA_CERVIX'].min(), HELA_express['HELA_CERVIX'].max()
HELA_express = HELA_express.sort_values(by =['Unnamed: 0']).set_index('Unnamed: 0')

In [ ]:
# look up cell line
def cell_line_lookup(filename):
    if 'CP186' in filename:
        cell_line = 'A549'
    if 'CP257' in filename:
        cell_line = 'HeLa'
    return cell_line

# look up condition for screen
def condition_lookup(filename, cell_line):
    if cell_line == 'HeLa':
        if 'DMEM' in filename:
            condition = 'DMEM'
        if 'HPLM' in filename:
            condition = 'HPLM'
    else: 
        condition = False
    return condition

In [3]:
# Change the feature names into integer values to speed computations
def features_to_ints(df):
    features = list(df.columns)[2:]
    features_dic_forward = {features[i] : i for i in range(len(features))}
    features_dic_reverse = {i : features[i] for i in range(len(features))}
    features[:10]
    df_int_feats = df.rename(columns=features_dic_forward)
    df_str_feats = df_int_feats.rename(columns=features_dic_reverse)
    return df_int_feats

,Metadata_Foci_Barcode_MatchedTo_GeneCode,Metadata_Foci_Barcode_MatchedTo_Barcode,Cells_AreaShape_CentralMoment_0_1,Cells_AreaShape_CentralMoment_0_3,Cells_AreaShape_CentralMoment_1_0,Cells_AreaShape_CentralMoment_1_2,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,Cells_AreaShape_HuMoment_1,...,Nuclei_Texture_SumVariance_ConA_5_01_256,Nuclei_Texture_SumVariance_DAPI_Painting_10_01_256,Nuclei_Texture_SumVariance_DAPI_Painting_10_03_256,Nuclei_Texture_SumVariance_Mito_5_01_256,Nuclei_Texture_SumVariance_Phalloidin_10_01_256,Nuclei_Texture_SumVariance_Phalloidin_10_03_256,Nuclei_Texture_SumVariance_Phalloidin_5_01_256,Nuclei_Texture_SumVariance_WGA_10_01_256,Nuclei_Texture_SumVariance_WGA_10_03_256,Nuclei_Texture_SumVariance_WGA_5_03_256
0,A1BG,CAAGAGAAAGACCACGAGCA,-0.085682,-0.007318,0.048388,0.005510,-0.029950,0.452470,0.128870,-0.251250,...,0.26915,-0.510610,-0.813530,-0.037740,-0.090246,-0.078726,-0.084498,-0.013743,0.06056,-0.093787
1,A1BG,CATCTTCTTTCACCTGAACG,0.009774,-0.003006,0.049408,0.006071,0.059289,-0.732130,-0.477380,-0.192550,...,-0.36389,-0.159780,-0.334250,-0.028623,-0.190650,-0.066117,-0.126050,-0.298190,-0.26774,-0.203280
2,A1BG,CTCCGGGGAGAACTCCGGCG,-0.058185,0.016190,-0.128340,-0.010445,-0.198250,0.118750,0.325120,-0.185930,...,-0.55675,0.168100,-0.094460,-0.088934,-0.256140,-0.121030,-0.277940,-0.344150,-0.29566,-0.349900
3,A1BG,TGGAAGTCCACTCCACTCAG,-0.097838,-0.008437,0.116660,0.004938,0.229590,0.605420,0.599460,-0.172990,...,-0.46291,0.406220,0.103390,-0.082164,-0.243800,-0.209550,-0.144330,-0.347550,-0.34362,-0.309940
4,A1CF,AGTTATGTTAGGTATACCCG,-0.057318,-0.029085,-0.034795,-0.026508,0.422990,0.374530,0.081628,0.045344,...,-0.28838,-0.023266,0.435720,-0.002229,0.076596,0.139300,0.128120,-0.069559,-0.23150,-0.108940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80855,nontargeting,TTTATGCATTTAATACGCCG,0.015472,-0.008321,-0.013841,-0.000692,0.004790,-0.049257,0.126070,-0.260380,...,-0.36470,-0.153800,-0.143790,-0.074116,-0.214040,-0.230090,-0.164120,-0.281430,-0.27678,-0.241060
80856,nontargeting,TTTCTAGTTACTACTGGACG,0.003309,-0.000789,-0.006572,-0.001090,-0.008685,-0.015765,0.135290,-0.221200,...,-0.40125,-0.074483,-0.066195,-0.055062,-0.203310,-0.222460,-0.201870,-0.243910,-0.24811,-0.195830
80857,nontargeting,TTTGGCAGTACCTTTTATTA,0.008112,-0.001499,-0.002163,0.000775,0.120530,0.019983,0.098362,-0.156850,...,-0.37106,-0.139890,-0.156200,-0.078060,-0.213080,-0.223870,-0.187550,-0.268350,-0.25324,-0.215150
80858,nontargeting,TTTTACCTTGTTCACATGGA,-0.004330,-0.004018,0.004450,-0.001802,0.175830,-0.049480,0.092460,-0.161920,...,-0.34700,-0.159690,-0.176560,-0.065137,-0.185410,-0.184750,-0.142810,-0.236550,-0.24324,-0.192800


In [5]:
# Extract the expression data from the Broad Institute Dependency Map data set
def describe_expression(df, cell_line):
    all_genes_list = list(df.Metadata_Foci_Barcode_MatchedTo_GeneCode.unique())
    if cell_line == 'A549':
        zero_express = A549_express.query('A549_LUNG == 0')
    if cell_line == 'HeLa':
        zero_express = HELA_express.query('HELA_CERVIX == 0')
    zero_express_list = list(zero_express.index)

    zero_tpm = [value for value in zero_express_list if value in all_genes_list]
    zero_tpm = sorted(list(set(zero_tpm)))

    expressed_gene_list = [value for value in all_genes_list if value not in zero_tpm]
    expressed_gene_list.remove('nontargeting')

    print(f"There are {len(zero_express_list)} unexpressed genes in this dataset")
    print(f"There are {len(all_genes_list)} total genes in this dataset")
    print(f"There are {len(zero_tpm)} ??? in this dataset")
    print(f"There are {len(expressed_gene_list)} expressed genes in this dataset")


,A549_LUNG
Unnamed: 0,
A1BG,2.459432
A1BG-AS1,1.378512
A1CF,0.176323
A2M,0.000000
A2M-AS1,0.464668
...,...
ZYG11AP1,0.000000
ZYG11B,2.648465
ZYX,6.538693


In [8]:
# Function to calculate U statistics and p-values using Mann-Whitney U test for each feature
def calculate_stats(feature_list, gene_list, df_genes, p_value):
    df_p_values_feature = pd.DataFrame(index=gene_list,columns=feature_list)
    df_u_values_feature = pd.DataFrame(index=gene_list,columns=feature_list)
    
    gene_counter = 0
    
    for feat in feature_list:
        list_p = []
        list_u = []
        for gene in gene_list: 
            gene_counter += 1
            if gene_counter % len(gene_list) == 0:
                print(f'now calculating feature number {gene_counter/len(gene_list)}')
        
            u, p = ss.mannwhitneyu(df_genes.query('Metadata_Foci_Barcode_MatchedTo_GeneCode == "nontargeting"')[feat],
                    df_genes.query('Metadata_Foci_Barcode_MatchedTo_GeneCode == @gene')[feat])
        
            list_p.append(p)
            list_u.append(u)
        
        df_p_values_feature[feat] = list_p
        df_u_values_feature[feat] = list_u
    
    df_u_values_feature = df_u_values_feature.apply(pd.to_numeric)
    df_p_values_feature = df_p_values_feature.apply(pd.to_numeric)
    
    df_p_values_feature.loc['sig_gene_count'] = 0
    
    for i in range(len(df_p_values_feature.columns)):
        count = 0
        for j in range(len(df_p_values_feature.index)-1):
            if df_p_values_feature.iloc[j,i] <= p_value:
                count +=1
        df_p_values_feature.iloc[len(df_p_values_feature.index)-1,i] = count

    return df_u_values_feature , df_p_values_feature


In [ ]:
# create hit lists for each file in file_list
# we created our hit lists from guide level normalized feature-selected median-aggregated profiles
# download files as described in README before performing

for profile_file in file_list:
    print (f"Now loading {profile_file}")
    df = pd.read_csv(os.path.join(input_folder,profile_file))
    df = df[df["Metadata_Foci_Barcode_MatchedTo_Barcode"].isin(guide_list)]
    
    print ("Converting feature strings to ints")
    df_int_feats = features_to_ints(df)

    cell_line = cell_line_lookup(profile_file)
    condition = condition_lookup(profile_file, cell_line)
    
    describe_expression(df, cell_line)

    df_genes = df_int_feats.query("Metadata_Foci_Barcode_MatchedTo_GeneCode != 'nontargeting'")
    genes = list(df_genes.Metadata_Foci_Barcode_MatchedTo_GeneCode.unique())
    features_int = list(df_genes.columns)[2:]
    features_int[:10]

    # Perform the statistics calculations for each feature
    df_u_values , df_p_values = calculate_stats(features_int, genes, df_int_feats, p_value)
    if condition:
        df_p_values.to_csv(os.path.join(output_folder,f'{cell_line}_{condition}_significant_features_mann_whitney_p_values.csv'))
        df_u_values.to_csv(os.path.join(output_folder,f'{cell_line}_{condition}_significant_features_mann_whitney_u_values.csv'))
    else:
        df_p_values.to_csv(os.path.join(output_folder,f'{cell_line}_significant_features_mann_whitney_p_values.csv'))
        df_u_values.to_csv(os.path.join(output_folder,f'{cell_line}_significant_features_mann_whitney_u_values.csv'))

    # Plot distribution
    plt.clf()
    data = df_p_values.loc['sig_gene_count']
    fig, ax = plt.subplots(1,1,figsize=(8,6))

    ax = sns.histplot(data)
    ax.set_ylabel('Significant features')
    ax.set_xlabel('Genes')
    plt.savefig(os.path.join(output_folder,'{cell_line}.png'), dpi=1200)
    plt.show()